In [ ]:
import sys
import hashlib

sys.path.insert(0, '../')
import util
from test_framework.key import ECKey, ECPubKey, generate_musig_key, generate_schnorr_nonce, aggregate_schnorr_nonces, aggregate_musig_signatures

# 1.2 Introduction to n-of-n MuSig

* Part 1: Public Key Generation.
* Part 2: Nonce Generation.
* Part 3: Signature Generation.

In this chapter, we introduce the interactive MuSig protocol which allows n-of-n participants to jointly create and spend taproot or tapscript output using aggregated Schnorr signatures. 

**The advantage of MuSig is both output and joint n-of-n spend are indistinguishable from other pay-to-pubkey transactions on-chain.**

MuSig is is an interactive protocol which requires multiple rounds of communication between the wallets. We will proceed with an example execution of the MuSig protocol below.

![test](../images/musig_intro_0.jpg)


## Part 1: Public Key Generation

* Due to linear nature of Schnorr, public keys, nonces and signatures can be aggregated.
* Due to the [Key cancellation](https://tlu.tarilabs.com/cryptography/digital_signatures/introduction_schnorr_signatures.html#key-cancellation-attack) attack, signers must first commit to individual public keys (Challenge factor).
* Individual key pairs must be multiplied with respective challenge.


![test](../images/musig_intro_1.jpg)

### 1.1 Programming Exercise: Compute 3-of-3 MuSig public key.
* Use `generate_musig_key([ECPubKey0, ECPubKey1, ...])`.
* returns `c_map, pk_musig`
* `c_map` contains `ECPubKey_i, challenge_data_i` key - value pairs.


In [ ]:
# Compute key pairs.
privkey0 = ECKey()
privkey1 = ECKey()
privkey2 = ECKey()
privkey0.generate()
privkey1.generate()
privkey2.generate()
pk0 = privkey0.get_pubkey()
pk1 = privkey1.get_pubkey()
pk2 = privkey2.get_pubkey()
pk_v = [pk0, pk1, pk2]

# Compute Key Challenges.
c_map, pk_musig = generate_musig_key(pk_v)
print("Aggregated Public Key: ", pk_musig.get_bytes().hex())
for pk, c in c_map.items():
    print("Compressed Pubkey: ", pk.get_bytes().hex())
    print("Challenge: ", c.hex(), '\n')

# Multiply key pairs by challenge factor.
privkey0_c = privkey0.mul(c_map[pk0])
privkey1_c = privkey1.mul(c_map[pk1])
privkey2_c = privkey2.mul(c_map[pk2])
pk0_c = pk0.mul(c_map[pk0])
pk1_c = pk1.mul(c_map[pk1])
pk2_c = pk2.mul(c_map[pk2])


## Part 2: Nonce Generation

* Nonces can be aggregated like public keys and signatures.
* Security [Proof](https://eprint.iacr.org/2018/068.pdf) requires nonces are randomly generated independently of each other.
* An initial nonce commitment round is necessary to ensure no nonce is derived as a function of another.


![test](../images/musig_intro_2.jpg)


### 2.1 Programming Exercise: Compute 3-of-3 MuSig nonce.

* `generate_schnorr_nonce()` returns a ECkey with with a jacobi symbol of 1.
* `aggregate_schnorr_nonces()` returns a ECPubKey and a boolean (indicates nonce negation).
* *Negation of nonce `k_i` isn't necessary, since it is performed by `sign_musig` in next exercise.*

In [ ]:
# Generate nonces.
k0 = generate_schnorr_nonce()
k1 = generate_schnorr_nonce()
k2 = generate_schnorr_nonce()
R0 = k0.get_pubkey()
R1 = k1.get_pubkey()
R2 = k2.get_pubkey()

# Skip nonce point commitment round (we assume nonce commitments are correct).

# Aggregate nonces.
R_agg, negated = aggregate_schnorr_nonces([R0, R1, R2])
print("R aggregated: ", R_agg.get_bytes().hex())
print("R_agg was negated: ", negated)


## Part 3: Signature Generation

* Signatures are generated individually, but can be aggregated once they are exchanged amongst all participants in a single roundtrip.
* Notice that the hash expressions are identical in all signatures, which makes aggregation possible.

![test](../images/musig_intro_3.jpg)

### 3.1 Programming exercise: Compute aggregated MuSig signature.

* `ECKey.sign_musig(nonce_point, negated, nonce_point_aggregated, pk_musig, msg)` returns individual signature.
* `aggregate_musig_signatures([sig0, sig1, ...])` returns aggregated signature.
* `ECPubKey.verify(sig, msg)` returns validity of signature.


In [ ]:
msg = hashlib.sha256(b'transaction').digest()

# Generate individual signatures.
sig0 = privkey0_c.sign_musig(k0, negated, R_agg, pk_musig, msg)
sig1 = privkey1_c.sign_musig(k1, negated, R_agg, pk_musig, msg)
sig2 = privkey2_c.sign_musig(k2, negated, R_agg, pk_musig, msg)

# Aggregate signatures.
sig_agg = aggregate_musig_signatures([sig0, sig1, sig2])
print("Signature verifies against MuSig pubkey: ", pk_musig.verify_schnorr(sig_agg, msg))
